# Task Description
從行政院環境環保署空氣品質監測網下載的觀測的資料  
**Expect: 實作 linear regression 預測出 PM2.5 的數值。**


# Assignment Regulation
+ hw1.sh
  + 請手刻實作 linear regression，方法限使用 gradient descent。  
  + 禁止使用 numpy.linalg.lstsq  
+ hw1_best.sh
  + 不限定作法，但套件規定仍必須遵照期初公告。

https://www.youtube.com/watch?v=Ye018rCVvOo&ab_channel=Hung-yiLee  
https://www.kaggle.com/competitions/ml2020spring-hw1/code

In [33]:
import numpy as np 
import pandas as pd
import seaborn as sns
import seaborn.objects as so
import plotly.express as px
import matplotlib.pyplot as plt


In [34]:
# csv
train_df = pd.read_csv("train.csv", encoding='utf-8')
test_df = pd.read_csv("test.csv", encoding='utf-8',
                    names = ['id', '測項', '0', '1', '2', '3', '4', '5', '6', '7', '8'])

# result
train_df

,日期,測站,測項,0,1,2,3,4,5,6,...,14,15,16,17,18,19,20,21,22,23
0,2014/1/1,豐原,AMB_TEMP,14,14,14,13,12,12,12,...,22,22,21,19,17,16,15,15,15,15
1,2014/1/1,豐原,CH4,1.8,1.8,1.8,1.8,1.8,1.8,1.8,...,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8
2,2014/1/1,豐原,CO,0.51,0.41,0.39,0.37,0.35,0.3,0.37,...,0.37,0.37,0.47,0.69,0.56,0.45,0.38,0.35,0.36,0.32
3,2014/1/1,豐原,NMHC,0.2,0.15,0.13,0.12,0.11,0.06,0.1,...,0.1,0.13,0.14,0.23,0.18,0.12,0.1,0.09,0.1,0.08
4,2014/1/1,豐原,NO,0.9,0.6,0.5,1.7,1.8,1.5,1.9,...,2.5,2.2,2.5,2.3,2.1,1.9,1.5,1.6,1.8,1.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4315,2014/12/20,豐原,THC,1.8,1.8,1.8,1.8,1.8,1.7,1.7,...,1.8,1.8,2,2.1,2,1.9,1.9,1.9,2,2
4316,2014/12/20,豐原,WD_HR,46,13,61,44,55,68,66,...,59,308,327,21,100,109,108,114,108,109
4317,2014/12/20,豐原,WIND_DIREC,36,55,72,327,74,52,59,...,18,311,52,54,121,97,107,118,100,105
4318,2014/12/20,豐原,WIND_SPEED,1.9,2.4,1.9,2.8,2.3,1.9,2.1,...,2.3,2.6,1.3,1,1.5,1,1.7,1.5,2,2


# 資料探索 
又稱探索性資料分析（Exploratory Data Analysis, EDA）
1. 構思關於這筆資料的問題。
2. 透過視覺化、轉換和建模，試著尋找答案。
3. 經由所得的結果，重整原先的問題，或者構思、增加新的問題。  


[R學習筆記：探索性資料分析](https://bookdown.org/b08302310/R_learning_notes/%E6%8E%A2%E7%B4%A2%E6%80%A7%E8%B3%87%E6%96%99%E5%88%86%E6%9E%90.html)

In [35]:
# 共有多少個測試項目
print("測試項目:\n", train_df['測項'].unique())
print("total label:", len(train_df['測項'].unique()))

# train: 整理資料
processed_train_df = train_df.copy()
processed_train_df[processed_train_df == 'NR'] = 0 # NR 都改成 0
processed_train_df = processed_train_df.drop(['測站'], axis=1)

# test: 整理資料
processed_test_df = test_df.copy()
processed_test_df[processed_test_df == 'NR'] = 0 # NR 都改成 0

processed_test_df
# processed_train_df


測試項目:
 ['AMB_TEMP' 'CH4' 'CO' 'NMHC' 'NO' 'NO2' 'NOx' 'O3' 'PM10' 'PM2.5'
 'RAINFALL' 'RH' 'SO2' 'THC' 'WD_HR' 'WIND_DIREC' 'WIND_SPEED' 'WS_HR']
total label: 18


,id,測項,0,1,2,3,4,5,6,7,8
0,id_0,AMB_TEMP,21,21,20,20,19,19,19,18,17
1,id_0,CH4,1.7,1.7,1.7,1.7,1.7,1.7,1.7,1.7,1.8
2,id_0,CO,0.39,0.36,0.36,0.4,0.53,0.55,0.34,0.31,0.23
3,id_0,NMHC,0.16,0.24,0.22,0.27,0.27,0.26,0.27,0.29,0.1
4,id_0,NO,1.3,1.3,1.3,1.3,1.4,1.6,1.2,1.1,0.9
...,...,...,...,...,...,...,...,...,...,...,...
4315,id_239,THC,1.8,1.8,1.8,1.8,1.7,1.7,1.7,1.7,1.7
4316,id_239,WD_HR,80,92,95,95,96,97,96,96,84
4317,id_239,WIND_DIREC,76,99,93,97,93,94,98,97,65
4318,id_239,WIND_SPEED,2.2,3.2,2.5,3.6,5,4.2,5.7,4.9,3.6


In [36]:
# 每個月拆開來看
# 20(day) * 18(features) = 360(records) * 12(month) = 4320(rows)

# train: 行、列交換一下
train_data_fixed = pd.DataFrame()
for i in range(24):
    train_data_slice = processed_train_df[['日期', '測項', str(i)]].copy()
    train_data_slice['日期_時間'] = pd.to_datetime(train_data_slice['日期'] + ' ' + str(i) +':00:00')
    train_data_slice['日期'] = train_data_slice['日期']
    train_data_slice['時'] = i
    train_data_slice = train_data_slice.pivot(index=[ '日期', '時'], columns=['測項'], values=str(i))
    train_data_fixed = pd.concat([train_data_fixed, train_data_slice])
    
# test: 行、列交換
test_data_fixed = pd.DataFrame()
for i in range(9):
    test_data_slice = processed_test_df[['id', '測項', str(i)]].copy()
    test_data_slice['id'] = test_data_slice['id'] + ' ' + str(i) + ':00:00'
    test_data_slice = test_data_slice.pivot(index='id', columns='測項', values=str(i))
    test_data_fixed = pd.concat([test_data_fixed, test_data_slice])
    
train_data_fixed.reset_index(inplace=True)
test_data_fixed.reset_index(inplace=True)

train_data_fixed
# test_data_fixed


測項,日期,時,AMB_TEMP,CH4,CO,NMHC,NO,NO2,NOx,O3,PM10,PM2.5,RAINFALL,RH,SO2,THC,WD_HR,WIND_DIREC,WIND_SPEED,WS_HR
0,2014/1/1,0,14,1.8,0.51,0.2,0.9,16,17,16,56,26,0,77,1.8,2,37,35,1.4,0.5
1,2014/1/10,0,12,1.8,0.28,0.05,1.4,3.8,5.2,27,5,6,0,73,1.2,1.8,73,63,2.4,1.4
2,2014/1/11,0,13,1.8,0.41,0.17,1.6,9.3,11,19,32,25,0,84,0.4,2,106,104,1.4,1.3
3,2014/1/12,0,17,1.8,0.5,0.19,1.8,16,17,29,87,48,0,87,4.1,2,176,179,1.7,1.6
4,2014/1/13,0,15,1.8,0.27,0.06,1.1,5.4,6.5,31,13,10,0,81,-0.3,1.8,30,350,1.4,0.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5755,2014/9/5,23,27,1.7,0.27,0.11,0.9,8.4,9.3,30,39,22,0,82,2.2,1.8,124,102,0.9,1.2
5756,2014/9/6,23,27,1.8,0.47,0.19,2,18,20,5.6,64,20,0,86,2.8,1.9,179,191,2.8,2.8
5757,2014/9/7,23,27,1.7,0.25,0.06,1.3,5.6,6.9,27,47,24,0,81,2.5,1.7,124,118,1.6,1.9
5758,2014/9/8,23,25,1.7,0.19,0.06,1.6,6.9,8.5,15,17,11,0,85,1.5,1.8,111,112,1.6,1.7


In [37]:
x_train_np = []
y_train_np = []
for row_date in train_data_fixed['日期'].unique():
    row_date_df = train_data_fixed[(train_data_fixed['日期'] == row_date)]
    prev_PM2dot5 = row_date_df.iloc[0]['PM2.5']

    x_train_temp_np = []
    y_train_temp_np = []
    PM2dot5 = 0
    count = 0
    for row_hour in row_date_df['時'].unique():
        row_hour_df = row_date_df[(row_date_df['時'] == row_hour)]
        
        if count != len(row_date_df) and count != 0:
            y_train_np.append(PM2dot5)
        
        PM2dot5 = row_hour_df.iloc[0]['PM2.5']

        # Add an extra column to the dataframe: prev_PM2.5
        row_hour_df['prev_PM2.5'] = row_hour_df.iloc[0]['PM2.5']
        prev_PM2dot5 = PM2dot5
        
        
        # x_train_temp_np = np.concatenate((x_train_temp_np, row_hour_df.iloc[0][2:].to_numpy()), axis=0)
        x_train_temp_np = row_hour_df.iloc[0][2:].to_numpy()
        x_train_np.append(x_train_temp_np)
        
        count += 1
        
    y_train_np.append(PM2dot5)
        
    # y_train_np.append(PM2dot5)
    # x_train_np.append(x_train_temp_np)

# 240(date) * 24(hour) * 18(feature)
# x_train_np
# y_train_np

x_test_np = []
prev_PM2dot5 = test_data_fixed.iloc[0]['PM2.5']
for row_date in test_data_fixed['id'].unique():
    row_date_df = test_data_fixed[(test_data_fixed['id'] == row_date)]
    
    # y train
    PM2dot5 = row_date_df.iloc[0]['PM2.5']

    # Add an extra column to the dataframe: prev_PM2.5
    row_date_df['prev_PM2.5'] = prev_PM2dot5
    prev_PM2dot5 = PM2dot5
    
    test_temp_np = row_date_df.iloc[0][1:].to_numpy()
    x_test_np.append(test_temp_np)

# 240(date) * 24(hour) * 18(feature)
x_train_np
# x_test_np

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


array(['17', '1.8', '0.56', '0.18', '2.1', '19', '21', '33', '85', '45',
       0, '72', '3.4', '2', '118', '119', '2.8', '2.5', '45'],
      dtype=object)

In [38]:
x_train_np = np.array(x_train_np, dtype=np.float16)
y_train_np = np.array(y_train_np, dtype=np.float16)

x_test_np = np.array(x_test_np, dtype=np.float16)

print("x_data shape:", x_train_np.shape)
print("y_data shape:", y_train_np.shape)

print("x test np", x_test_np.shape)

# mean = np.mean(train_np, axis=0)
# std = np.std(train_np, axis=0)

# print("mean shape:", mean.shape)
# print("std shape:", std.shape)


x_data shape: (5760, 19)
y_data shape: (5760,)
x test np (2160, 19)


In [39]:

# 定义成本函数
def compute_cost(X, y, w, b):
    m = len(y)
    y_pred = np.dot(X, w) + b
    cost = (1/(2*m)) * np.sum(np.square(y_pred - y))
    return cost


# 实现梯度下降算法
def gradient_descent(X, y, w, b, learning_rate, num_iterations):
    m = len(y)
    for i in range(num_iterations):
        y_pred = np.dot(X, w) + b
        dw = (1/m) * np.dot(X.T, (y_pred - y))
        db = (1/m) * np.sum(y_pred - y)
        w = w - learning_rate * dw
        b = b - learning_rate * db
        cost = compute_cost(X, y, w, b)
        if i % (num_iterations / 10) == 0:
            print("Cost after iteration {}: {}".format(i, cost))
    return w, b

# 初始化权重矩阵和偏置项
w = np.random.randn(19, 1)
b = 0

# 运行模型
X = np.array(x_train_np)
y = np.array(y_train_np).reshape(-1, 1)

print("X shape {}, y shpae {}".format(X.shape, y.shape))
w, b = gradient_descent(X, y, w, b, learning_rate=0.0000005, num_iterations=800000)

# 權重
print("w:", w)
print("b:", b)

# 进行预测
y_pred = np.dot(X, w) + b
print("y shape", y_pred.shape)
y_pred


X shape (5760, 19), y shpae (5760, 1)
Cost after iteration 0: 50210.29203865976
Cost after iteration 50000: 8.28603583582608
Cost after iteration 100000: 4.6718391489319195
Cost after iteration 150000: 3.5648361918970752
Cost after iteration 200000: 2.9756999581662376
Cost after iteration 250000: 2.5438282368887593


In [ ]:
w

array([[-9.32260314e-03],
       [ 1.12888005e+00],
       [-1.42552410e+00],
       [ 3.15189973e-01],
       [ 2.52666275e-01],
       [ 3.81318160e-01],
       [-3.42582352e-01],
       [-2.80881686e-03],
       [-2.88890480e-03],
       [ 9.95994523e-01],
       [ 1.45538247e-01],
       [-5.55532918e-03],
       [ 2.34066940e-01],
       [-7.83405807e-01],
       [-1.06271975e-03],
       [-1.44107137e-04],
       [ 1.20243771e-01],
       [-5.99928422e-02],
       [ 2.38911727e-03]])

In [ ]:
# plt_np = np.concatenate((y, y_pred), axis=1)
# plt_np.shape

plt_np = {
    # "ori_y": y.reshape(5760),
    "pred_y": y_pred.reshape(5760)
}
plt_np

plt_df = train_data_fixed[['日期', '時', 'PM2.5']]
plt_df['pred_y'] = y_pred

# 將日期和時間列轉換為datetime類型
plt_df['日期'] = pd.to_datetime(plt_df['日期'])
plt_df['時'] = pd.to_timedelta(plt_df['時'], unit='h')

# 創建新的日期＿時列
plt_df['日期_時'] = plt_df['日期'] + plt_df['時']
# 刪除原日期和時間列
plt_df.drop(['日期', '時'], axis=1, inplace=True)

# print(dtypes(plt_df['value']))

p_df = pd.melt(plt_df, id_vars=['日期_時'], value_vars=['PM2.5', 'pred_y'], var_name='type', value_name='value')
p_df["value"] = p_df["value"].astype(float)
p_df = p_df.sort_values('日期_時')
p_df
# print(p_df.dtypes)


# 显示图表
fig = px.line(p_df.head(300), x='日期_時', y='value', color='type')
fig.show()

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[

In [ ]:
y_test_pred = np.dot(x_test_np, w) + b
print(y_test_pred.shape)
y_test_pred


(2160, 1)


array([[33.20615956],
       [17.12721033],
       [47.91804462],
       ...,
       [12.93298731],
       [29.19972606],
       [ 1.38143146]])

# ----

In [ ]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression().fit(X, y)
reg.score(X, y)
reg.coef_
reg.intercept_
reg.predict(np.array([[3, 5]]))

ValueError: X has 2 features, but LinearRegression is expecting 19 features as input.